In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from IPython.display import display
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from IPython.core.display import display, HTML
from numpy import nan
display(HTML("<style>"
    + "#notebook { padding-top:0px; } " ""
    + ".container { width:100%; } "
    + ".end_space { min-height:0px; } "
    + "</style>"))
# import colors


def highlight_max(sett):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    is_max = s == s.max()
    
    return ['font-weight: bold' if v else '' for v in is_max]

def highlight_group(sett):
    try:
        if sett.dtype == 'O':
            oboe = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
        else:
            oboe = sett
        is_max = oboe.to_frame().apply(lambda x: oboe[oboe.index.get_level_values(0) == x.name[0]].max() == x.item(), axis=1)
        return ['font-weight: bold' if v else '' for v in is_max]
    except:
        return ['' for _ in sett]

def highlight_group_min(sett):
    try:
        if sett.dtype == 'O':
            oboe = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
        else:
            oboe = sett
        is_max = oboe.to_frame().apply(lambda x: oboe[oboe.index.get_level_values(0) == x.name[0]].min() == x.item(), axis=1)
        return ['font-weight: bold' if v else '' for v in is_max]
    except:
        return ['' for _ in sett]

    
def highlight_min(sett):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    is_max = s == s.min()
    
    return ['font-weight: bold' if v else '' for v in is_max]

def b_g(sett, cmap='PuBu', low=0, high=0):
    if sett.dtype == 'O':
        s = sett.str.split(' ').apply(lambda x: float(x[0]) if x[0] != '-' else np.nan)
    else:
        s = sett
    a = s
    rng = a.max() - a.min()
    norm = colors.Normalize(a.min() - (rng * low),
                        a.max() + (rng * high))
    normed = norm(a.values)
    c = [colors.rgb2hex(x) for x in plt.cm.get_cmap(cmap)(normed)]
    return ['background-color: %s' % color for color in c]

def method_to_df(apath, setting, dataset, method):
    last = ' '.join([setting, dataset, method])
    with open(apath + '%s/%s/' % (setting, dataset) + method +'/logs.pyd', 'r') as f:
        lines = f.read().splitlines()
    o = pd.DataFrame.from_records([eval(x.replace('nan', 'float(\'nan\')')) for x in lines])
    
    o = o.rename(columns={'cs_size':'buffer_size'})
    if 'buffer_size' not in o:
        o['buffer_size'] = 0
    if 'job_number' in o.columns:
        del o['job_number']
    o['method'] = method
    return o


req_rep = 1
print_args= False
display_pre = False
howmany_pre = 1
print_other_metrics = False
t_distinct_cols = ['buffer_size', 'method', 'shap_weight', 'shap_all', 'n_epochs']
ignore_cols = []
excluded_methods = []
included_methods = []
excluded_datasets = []

sec_respath = None

# PAMI
respath = '../storage/results/dataset-can-mnist/classes_per_task-1/logs.pyd'#'C:\\Users\\emace\\AImageLab\\SRV-Continual\\results\\mammoth\\results\\'
sec_respath = '../storage/results/dataset-can-mnist/classes_per_task-2/logs.pyd' #'/home/mbosc/srv-nas/mammoth-master/results/'
# third_respath = '../storage/results/dataset-can-sole/classes_per_task-1/logs.pyd'
third_respath = None #'/home/mbosc/phd/pami/results_buzz/'

respaths = [respath, sec_respath, third_respath]
islist = lambda col: any([type(x) == list for x in col.values])


In [ ]:
!../utils/download_results.sh /nas/softechict-nas-1/rbenaglia/canomaly-data/results/dataset-can-mnist/ ../storage/results/

In [ ]:
## import file
list_total_parsed = []
for path in respaths:
    if path is not None:
        with open(os.path.join(path), 'r') as obj:
            file = obj.readlines()
        file_parsed = map(eval, file)
        list_total_parsed.extend(list(file_parsed))
    
df_file = pd.DataFrame(list_total_parsed)
df_file['timestamp'] = pd.to_datetime(df_file['timestamp'])
df_file = df_file[df_file['timestamp']>'2022-02-02 16:50:00']
# df_file = df_file[df_file['timestamp']>'2022-02-05 16:50:00']
df_file.shape

In [ ]:
df_file.head()

In [ ]:
# total_auc = pd.concat([pd.DataFrame.from_dict(sing[1].loc[0,'auc_per_task'], orient='index')]*2)
# total_auc.reset_index().groupby('index').mean()

def calc_mean_on_matrix(X):
    total_df = pd.concat([pd.DataFrame.from_dict(x, orient='index') for x in X])
    return total_df.reset_index().groupby('index').mean().to_dict(orient='index')

In [ ]:
def calc_mean_pm_std(x):
    return f'{x.mean().round(3)}+-{x.std().round(3)}'

In [ ]:
# Converting links to html tags
def path_to_image_html(path):
    return '<img src="'+ path + '" width="60" >'

In [ ]:
df_elab = df_file.copy()

common_keys=['dataset','classes_per_task','model','optim','lr','batch_size','n_epochs','latent_space','joint']
part_keys = {'sae':['sparse_weight','norm_order'], 'ae':[], 'vae':['beta_kl'],'dae':['noise_mean','noise_std']}
elaboration_cols = {
    'auc_final':[calc_mean_pm_std, len, np.mean, np.std],
    'auc_average':[calc_mean_pm_std, np.mean, np.std],
    'auc_per_task': [calc_mean_on_matrix],
    'conf_matrix_per_task': [calc_mean_on_matrix],
    'id': [list]
    
}
task_per_task_elaboration_cols = {
    
}
total_list = []
for model, df_model in df_elab.groupby('model'):
    df_model['model'] = model
    total_keys = common_keys+part_keys[model]
    
    df_single = df_model.groupby(total_keys).agg(elaboration_cols)
        
    total_list.append(df_single.reset_index())

df_total = pd.concat(total_list).set_index(common_keys)
df_total.columns = ['_'.join(x) for x in df_total.columns.ravel()]

In [ ]:
def plot_heatmap(matrix_dict, title=None, cmap='Reds'):
    fig, ax = plt.subplots(figsize=(10,8))
    sns.heatmap(pd.DataFrame.from_dict(matrix_dict, orient='index'), annot=True, cbar=False, cmap=cmap, ax=ax, linewidths=0.1)
    ax.set_xlabel('Label')
    ax.set_ylabel('Task')
    if title is not None:
        ax.set_title(title)
def plot_heatmaps(df: pd.DataFrame, which_col=('auc_per_task', 'calc_mean_on_matrix')):
    for title,row in df.iterrows():
        plot_heatmap(row.loc[which_col], title='\n'.join(f'{x}: {y}' for x,y in zip(df.index.names,title)))

In [ ]:
def filter_on_df_total(df, sort_by_list = [],ascending=False, **kwargs):
    fil = ' and '.join([f'{k}=={v}' if isinstance(v,(int,float, bool)) else f'{k}=="{v}"' for k,v in kwargs.items()])
    df_sub = df.reset_index().query(fil).set_index(common_keys)
    return df_sub.sort_values(by=sort_by_list, ascending=ascending)

In [ ]:
kwargs = dict(joint=False,classes_per_task=2,)
df_filter = filter_on_df_total(df_total, 
                               sort_by_list=['auc_average_mean'], 
                               ascending=False, 
                               **kwargs)
df_filter

In [ ]:
# id_ = df_filter.iloc[1, df_filter.columns.to_list().index('id_list')][0]
id_ = '23b78d21-6ae2-4e28-9c64-b7edcb3d8632'
! ../utils/download_logs.sh /nas/softechict-nas-1/rbenaglia/canomaly-data/logs/dataset-can-mnist/classes_per_task-2/logs.pyd $id_
! ../utils/download_results.sh single_log.pyd ../storage/logs/single_log.pyd

In [ ]:
sns.heatmap(df_total.iloc[-2, -2])

In [ ]:
# gridmap(methods, ['3ea70747-8697-4e76-bc4f-319c6fa312b5', '89f4c672-f16a-4b08-a509-a94c82a6ebbd', '31d4cc8c-a727-4665-8643-2212eb519aa6', '26ea2435-d2d4-4988-b37e-d95934301d2f', 'a0626b3b-b34f-4353-8453-4abd769c130c', '9ae21984-b77f-4d12-be42-c4929c4d4125', '1cc0e452-8112-47ac-b939-f9ad5ae5edc6', '86d1bc60-d263-46be-b90f-ff9d9a30173a', '606653f0-b606-426c-9e8f-7770e8dad36a', '6ef54be8-cedf-40d7-a7c7-cda396b45d1c', '918ff2d4-b0d1-456d-96a1-e283e7838827'],
# ['57976485-efee-4aae-95e2-4a2ac112ac3e', 'ff4eb4f9-0e6b-4336-bd66-057a02d2f663', '590d6225-04f7-4ba6-b8c7-0182262ccdf5', '78b3b890-bcc0-4fa6-bc7a-2569f12cee23', '0c875241-6230-41d1-9875-09a2c4c586c0', 'f608773d-99e2-49c9-b1d2-6857966d6977', 'f9a60776-ec0b-47fd-8895-27743c5c9911', '74b41430-89b3-4c1c-a058-9e01bb05480a', '45679ee3-d077-4e5a-83cc-77d4670d27ea', '1e7573db-8821-4cd7-b243-3f4fe9d31553', 'ad3fa7bc-8a5a-4fa6-a2b1-5d3072597e02'])